In [28]:
import numpy as np
import cudamat as cm

n, p = int(2e3), int(40e3)
A = np.random.randn(n, p)
B = np.random.randn(p, n)
def op(A,B):
    return A @B
%timeit op(A,B)

1 loop, best of 3: 2.23 s per loop


In [12]:


cm.cublas_init()
cm.CUDAMatrix.init_random()
A_cm = cm.empty((n, p)).fill_with_randn()
B_cm = cm.empty((p, n)).fill_with_randn()
%timeit A_cm.dot(B_cm)
cm.cublas_shutdown()

1000 loops, best of 3: 528 µs per loop


In [6]:
import cython
A = np.random.randn(n, p)
B = np.random.randn(p, n)

%timeit  cython.inline("op(A,B)")

1 loop, best of 3: 2.26 s per loop


In [13]:
from numba import jit
@jit(nopython=True,nogil=True,cache=True)
def op(A,B):
    return A @B
%timeit op(A,B)

1 loop, best of 3: 2.2 s per loop


In [37]:
A=np.array((n,p),dtype=float32)
B=np.array((n,p),dtype=float32)
A = np.random.randn(n, p)
B = np.random.randn(p, n)
C=A.copy()

NameError: name 'float32' is not defined

In [36]:
from numba import cuda
TPB = 16

@cuda.jit
def matmul(A, B, C):
    """Perform square matrix multiplication of C = A * B
    """
    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]
        C[i, j] = tmp
%timeit fast_matmul(A,B,C)

TypingError: Failed at nopython (nopython frontend)
Untyped global name 'float32': cannot determine Numba type of <class 'object'>
File "<ipython-input-21-55614d056554>", line 8